In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix 
import itertools
import os
import shutil
import random 
import glob
import matplotlib.pyplot as plt 
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:

import zipfile
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip",'r') as zip_ref:
    zip_ref.extractall(".")
   

# !cp -r original_directory_path ./



In [ ]:
os.chdir("data/")

In [ ]:
!pwd

In [ ]:
if os.path.isdir('train/dog') is False: 
  os.makedirs('train/dog')
  os.makedirs('train/cat')
  os.makedirs('valid/dog')
  os.makedirs('valid/cat')
  os.makedirs('test/dog')
  os.makedirs('test/cat')
# train = pd.read_csv('/kaggle/input/dogs-vs-cats/train.zip')

In [ ]:
for c in random.sample(glob.glob('cat*'), 500): 
    shutil.move(c, 'train/cat')
for c in random.sample(glob.glob('dog*'), 500): 
    shutil.move(c, 'train/dog')
for c in random.sample(glob.glob('cat*'), 100): 
    shutil.move(c, 'valid/cat')
for c in random.sample(glob.glob('dog*'), 100): 
    shutil.move(c, 'valid/dog')
for c in random.sample(glob.glob('cat*'), 50): 
    shutil.move(c, 'test/cat')
for c in random.sample(glob.glob('dog*'), 50): 
    shutil.move(c, 'test/dog')

In [ ]:
!pwd


In [ ]:
train_path = 'train/'
test_path = 'test/'
valid_path =  'valid/'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=["cat", "dog"], batch_size=10)
valid_batches  = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=["cat", "dog"], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=["cat", "dog"], batch_size=10, shuffle = False)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
# this function will plot images in the form of a grid with 1 row and 10 columns where images are placed
def plotImages(images_arr): 
    fig, axes = plt.subplots(1, 10, figsize =  (20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes ): 
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(imgs)
print(labels)
# this is what a batch of training data looks like 

In [ ]:
model = Sequential([ 
    Conv2D(filters=32, kernel_size = (3,3), activation="relu", padding="same", input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2), strides = 2), 
    Conv2D(filters=64, kernel_size = (3,3), activation = "relu", padding = "same"), 
    MaxPool2D(pool_size=(2,2), strides = 2), 
    Flatten(), # flattened to one dimensional tensor
    Dense(units=2, activation = "softmax"), 
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.0001), loss = "categorical_crossentropy", metrics=["accuracy"]) 

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs= 10, verbose=2)


In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels) 
# all pics in batch will be same (like all cats or all dogs) because 
# we specified that we do not want to shuffle it earlier 

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
np.round(predictions)

In [ ]:
# confusion matrix can be created